# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))
from db_connection.connection import PostgresConnection

## Initialize and connect to the PostgreSQL database


In [2]:
db = PostgresConnection()
db.connect()

Connected to PostgreSQL Database!


## Define a query to fetch data from the xdr_data table

In [3]:
query = "SELECT * FROM xdr_data;"


# Fetch data into a DataFrame

In [4]:
df = db.fetch_data(query)


c:\Users\habteyes.asfaw\10Accadamy\TellCo-Telecom-User-Analytics\db_connection\connection.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)


### Display the first few rows of the fetched data


In [5]:

df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


# Close the connection


In [6]:
db.close()


Connection closed.


# Data Overview and Cleaning


## Display basic info and statistics


In [7]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150000.000000,150000.000000,1.500000e+05,1.494310e+05,1.489350e+05,1.494290e+05,122172.000000,122189.000000,150000.000000,...,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500000e+05,1.500000e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,...,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.611834,288.097653,8.103762e+04,2.148809e+10,2.447443e+12,2.241637e+13,619.782739,84.793524,23971.878541,...,6.710569e+06,6.345423e+06,6.725218e+06,6.359490e+06,2.439675e+08,4.782700e+06,2.432050e+08,4.769004e+06,1.127639e+07,2.441429e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,...,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744050e+04,2.082014e+14,3.365130e+10,3.546071e+13,32.000000,2.000000,43.000000,...,5.833501e+06,5.517965e+06,5.777156e+06,5.475981e+06,2.104733e+08,4.128476e+06,2.101869e+08,4.145943e+06,3.322201e+07,2.431068e+08
50%,7.349883e+18,499.000000,500.000000,8.639900e+04,2.082015e+14,3.366371e+10,3.572201e+13,45.000000,5.000000,63.000000,...,1.161602e+07,1.101345e+07,1.164222e+07,1.099638e+07,4.234081e+08,8.291208e+06,4.218030e+08,8.267071e+06,4.114331e+07,4.558411e+08
75%,1.304243e+19,749.000000,750.000000,1.324302e+05,2.082018e+14,3.368349e+10,8.611970e+13,70.000000,15.000000,19710.750000,...,1.744852e+07,1.651556e+07,1.747048e+07,1.650727e+07,6.331742e+08,1.243162e+07,6.316918e+08,1.238415e+07,4.903424e+07,6.657055e+08
max,1.318654e+19,999.000000,999.000000,1.859336e+06,2.140743e+14,8.823971e+14,9.900120e+13,96923.000000,7120.000000,378160.000000,...,2.325910e+07,2.201196e+07,2.325919e+07,2.201196e+07,8.434419e+08,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08


# Check for missing values

In [8]:
print("Missing values:\n", df.isnull().sum())


Missing values:
 Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps

## Check for outliers using boxplot for numeric columns


# Exploratory Data Analysis


## Aggregate per user


In [9]:
for idx, col in enumerate(df.columns):
    print(f" Column Name: '{col}'")


 Column Name: 'Bearer Id'
 Column Name: 'Start'
 Column Name: 'Start ms'
 Column Name: 'End'
 Column Name: 'End ms'
 Column Name: 'Dur. (ms)'
 Column Name: 'IMSI'
 Column Name: 'MSISDN/Number'
 Column Name: 'IMEI'
 Column Name: 'Last Location Name'
 Column Name: 'Avg RTT DL (ms)'
 Column Name: 'Avg RTT UL (ms)'
 Column Name: 'Avg Bearer TP DL (kbps)'
 Column Name: 'Avg Bearer TP UL (kbps)'
 Column Name: 'TCP DL Retrans. Vol (Bytes)'
 Column Name: 'TCP UL Retrans. Vol (Bytes)'
 Column Name: 'DL TP < 50 Kbps (%)'
 Column Name: '50 Kbps < DL TP < 250 Kbps (%)'
 Column Name: '250 Kbps < DL TP < 1 Mbps (%)'
 Column Name: 'DL TP > 1 Mbps (%)'
 Column Name: 'UL TP < 10 Kbps (%)'
 Column Name: '10 Kbps < UL TP < 50 Kbps (%)'
 Column Name: '50 Kbps < UL TP < 300 Kbps (%)'
 Column Name: 'UL TP > 300 Kbps (%)'
 Column Name: 'HTTP DL (Bytes)'
 Column Name: 'HTTP UL (Bytes)'
 Column Name: 'Activity Duration DL (ms)'
 Column Name: 'Activity Duration UL (ms)'
 Column Name: 'Dur. (ms).1'
 Column Name:

In [10]:
user_agg = df.groupby('MSISDN/Number').agg(
    num_sessions=('Bearer Id', 'count'),  
    total_duration=('Dur. (ms)', 'sum'),  
    total_dl=('Total DL (Bytes)', 'sum'),  
    total_ul=('Total UL (Bytes)', 'sum')  
).reset_index()


In [11]:
print("\nUser Aggregation Data:")
print(user_agg.head())


User Aggregation Data:
   MSISDN/Number  num_sessions  total_duration      total_dl    total_ul
0   3.360100e+10             1        116720.0  8.426375e+08  36053108.0
1   3.360100e+10             1        181230.0  1.207552e+08  36104459.0
2   3.360100e+10             1        134969.0  5.566597e+08  39306820.0
3   3.360101e+10             1         49878.0  4.019932e+08  20327526.0
4   3.360101e+10             2         37104.0  1.363130e+09  94280527.0


# Analysis of top 10 handsets


In [12]:
top_handsets = df['handset'].value_counts().head(10)
print("\nTop 10 Handsets:")
print(top_handsets)

KeyError: 'handset'